In [ ]:
# Initialize security lab environment

# Import core libraries for data generation and analysis
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

# Set base parameters for reproducibility
random.seed(42)
np.random.seed(42)

# Define core time window for security events
SIM_START_TIME = datetime(2025, 1, 1, 9, 0, 0)
SIM_EVENT_COUNT_BASELINE = 1000


In [ ]:
  # Generate baseline security event stream

# Define helper function to generate baseline security events
def generate_baseline_events(count, start_time):
    # Define typical users, IP ranges and actions
    users = ["alice", "bob", "carol", "dave"]
    base_ips = ["10.0.0.", "10.0.1.", "10.0.2."]
    actions = ["LOGIN_SUCCESS", "LOGOUT", "VIEW_DASHBOARD", "DOWNLOAD_REPORT"]

    # Create event list with benign behavior
    events = []
    current_time = start_time
    for i in range(count):
        current_time += timedelta(seconds=np.random.exponential(scale=30))
        user = random.choice(users)
        ip = random.choice(base_ips) + str(random.randint(2, 250))
        action = random.choice(actions)
        events.append(
            {"timestamp": current_time, "user": user, "ip": ip, "action": action}
        )

    return pd.DataFrame(events)

# Create baseline event DataFrame
df_events = generate_baseline_events(SIM_EVENT_COUNT_BASELINE, SIM_START_TIME)
df_events.head()


,timestamp,user,ip,action
0,2025-01-01 09:00:14.078043,alice,10.0.0.191,VIEW_DASHBOARD
1,2025-01-01 09:01:44.381686,bob,10.0.0.37,LOGIN_SUCCESS
2,2025-01-01 09:02:23.884057,alice,10.0.2.110,LOGIN_SUCCESS
3,2025-01-01 09:02:51.272334,alice,10.0.0.57,LOGOUT
4,2025-01-01 09:02:56.361080,alice,10.0.2.52,DOWNLOAD_REPORT


In [ ]:
# Generate red team login attack events

# Define helper function to model brute force style activity
def generate_bruteforce_events(count, start_time):
    # Define attacker ip and target users
    attacker_ip = "203.0.113.10"
    target_users = ["alice", "bob", "carol"]
    actions = ["LOGIN_FAILURE", "LOGIN_LOCKED"]

    # Create list of repeated login attempts
    events = []
    current_time = start_time + timedelta(minutes=10)
    for i in range(count):
        current_time += timedelta(seconds=np.random.exponential(scale=5))
        user = random.choice(target_users)
        action = np.random.choice(actions, p=[0.9, 0.1])
        events.append(
            {"timestamp": current_time, "user": user, "ip": attacker_ip, "action": action}
        )

    return pd.DataFrame(events)

# Create attack event DataFrame
df_attacks = generate_bruteforce_events(200, SIM_START_TIME)
df_attacks.head()


,timestamp,user,ip,action
0,2025-01-01 09:10:01.023651,carol,203.0.113.10,LOGIN_FAILURE
1,2025-01-01 09:10:11.339360,carol,203.0.113.10,LOGIN_FAILURE
2,2025-01-01 09:10:19.553329,bob,203.0.113.10,LOGIN_FAILURE
3,2025-01-01 09:10:25.446098,bob,203.0.113.10,LOGIN_FAILURE
4,2025-01-01 09:10:26.882296,alice,203.0.113.10,LOGIN_FAILURE


In [ ]:
# Merge baseline and attack events and add security labels

# Concatenate baseline and attack streams
df_security_events = pd.concat([df_events, df_attacks], ignore_index=True)

# Sort by timestamp for chronological analysis
df_security_events = df_security_events.sort_values("timestamp").reset_index(drop=True)

# Add column with simple role perspective
def assign_role(ip):
    # Assign red team tag to known attacker ip
    if ip == "203.0.113.10":
        return "red_team"
    return "normal_activity"

df_security_events["role"] = df_security_events["ip"].apply(assign_role)

# Add basic label for suspicious vs benign behavior
df_security_events["label"] = np.where(
    (df_security_events["role"] == "red_team")
    & (df_security_events["action"].isin(["LOGIN_FAILURE", "LOGIN_LOCKED"])),
    "suspicious",
    "benign",
)

df_security_events.head(15)


,timestamp,user,ip,action,role,label
0,2025-01-01 09:00:14.078043,alice,10.0.0.191,VIEW_DASHBOARD,normal_activity,benign
1,2025-01-01 09:01:44.381686,bob,10.0.0.37,LOGIN_SUCCESS,normal_activity,benign
2,2025-01-01 09:02:23.884057,alice,10.0.2.110,LOGIN_SUCCESS,normal_activity,benign
3,2025-01-01 09:02:51.272334,alice,10.0.0.57,LOGOUT,normal_activity,benign
4,2025-01-01 09:02:56.361080,alice,10.0.2.52,DOWNLOAD_REPORT,normal_activity,benign
5,2025-01-01 09:03:01.448969,bob,10.0.1.152,VIEW_DASHBOARD,normal_activity,benign
6,2025-01-01 09:03:03.244132,alice,10.0.0.180,DOWNLOAD_REPORT,normal_activity,benign
7,2025-01-01 09:04:03.581058,carol,10.0.1.41,LOGOUT,normal_activity,benign
8,2025-01-01 09:04:31.153523,carol,10.0.0.25,DOWNLOAD_REPORT,normal_activity,benign
9,2025-01-01 09:05:08.091025,alice,10.0.1.218,VIEW_DASHBOARD,normal_activity,benign


In [ ]:
# Analyze login failures with SIEM-style aggregation

# Import additional library for numerical operations if needed
import numpy as np

# Ensure timestamp column be datetime
df_security_events["timestamp"] = pd.to_datetime(df_security_events["timestamp"])

# Create 5-minute buckets for correlation, similar to Splunk or Elastic dashboards
df_security_events["time_bucket_5min"] = df_security_events["timestamp"].dt.floor("5min")

# Filter login-related actions often monitored by SIEM and IDS tools
login_events = df_security_events[
    df_security_events["action"].isin(["LOGIN_FAILURE", "LOGIN_LOCKED"])
]

# Aggregate failure counts per ip and time bucket
agg_bruteforce = (
    login_events.groupby(["ip", "time_bucket_5min"])
    .size()
    .reset_index(name="failure_count")
)

# Define threshold inspired by brute force detection rules in tools like Fail2ban or WAF appliances
THRESHOLD_FAILURES_PER_5MIN = 20

# Flag buckets that exceed the threshold
agg_bruteforce["alert_bruteforce"] = np.where(
    agg_bruteforce["failure_count"] >= THRESHOLD_FAILURES_PER_5MIN, 1, 0
)

agg_bruteforce.head()


,ip,time_bucket_5min,failure_count,alert_bruteforce
0,203.0.113.10,2025-01-01 09:10:00,44,1
1,203.0.113.10,2025-01-01 09:15:00,53,1
2,203.0.113.10,2025-01-01 09:20:00,64,1
3,203.0.113.10,2025-01-01 09:25:00,39,1


In [ ]:
# Generate blue team alerts and recommended actions

# Merge aggregated alerts back into main event stream
df_security_events = df_security_events.merge(
    agg_bruteforce[["ip", "time_bucket_5min", "alert_bruteforce"]],
    on=["ip", "time_bucket_5min"],
    how="left",
)

# Fill missing values with zero for periods without alerts
df_security_events["alert_bruteforce"] = df_security_events["alert_bruteforce"].fillna(0)

# Define helper function to suggest actions similar to SIEM, SOAR or firewall tools
def suggest_blue_team_action(row):
    # Suggest block or rate limit for attacker ip after alert, similar to Fail2ban, Palo Alto or Fortinet firewalls
    if row["alert_bruteforce"] == 1 and row["role"] == "red_team":
        return "block_ip_and_force_password_reset"
    # Suggest monitor for normal activity
    return "monitor"

# Apply blue team action suggestion to each event
df_security_events["blue_team_action"] = df_security_events.apply(
    suggest_blue_team_action, axis=1
)

df_security_events.head(20)


,timestamp,user,ip,action,role,label,time_bucket_5min,alert_bruteforce,blue_team_action
0,2025-01-01 09:00:14.078043,alice,10.0.0.191,VIEW_DASHBOARD,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
1,2025-01-01 09:01:44.381686,bob,10.0.0.37,LOGIN_SUCCESS,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
2,2025-01-01 09:02:23.884057,alice,10.0.2.110,LOGIN_SUCCESS,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
3,2025-01-01 09:02:51.272334,alice,10.0.0.57,LOGOUT,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
4,2025-01-01 09:02:56.361080,alice,10.0.2.52,DOWNLOAD_REPORT,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
5,2025-01-01 09:03:01.448969,bob,10.0.1.152,VIEW_DASHBOARD,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
6,2025-01-01 09:03:03.244132,alice,10.0.0.180,DOWNLOAD_REPORT,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
7,2025-01-01 09:04:03.581058,carol,10.0.1.41,LOGOUT,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
8,2025-01-01 09:04:31.153523,carol,10.0.0.25,DOWNLOAD_REPORT,normal_activity,benign,2025-01-01 09:00:00,0.0,monitor
9,2025-01-01 09:05:08.091025,alice,10.0.1.218,VIEW_DASHBOARD,normal_activity,benign,2025-01-01 09:05:00,0.0,monitor


In [ ]:
# Build engineered features for anomaly detection

# Create aggregate metrics per ip and time window for UEBA-style analysis
group_cols = ["ip", "time_bucket_5min"]

agg_features = df_security_events.groupby(group_cols).agg(
    total_events=("action", "count"),
    login_failures=("action", lambda x: (x == "LOGIN_FAILURE").sum()),
    login_locked=("action", lambda x: (x == "LOGIN_LOCKED").sum()),
    distinct_users=("user", "nunique"),
    suspicious_events=("label", lambda x: (x == "suspicious").sum()),
    bruteforce_alerts=("alert_bruteforce", "sum"),
).reset_index()

# Compute ratios often used in SIEM, XDR and UEBA analytics
agg_features["failure_ratio"] = agg_features["login_failures"] / agg_features["total_events"]
agg_features["suspicious_ratio"] = agg_features["suspicious_events"] / agg_features["total_events"]

# Add high risk label for training supervised and unsupervised models
def label_bucket(row):
    # Mark bucket as high risk when suspicious events or brute force alerts are present
    if row["suspicious_events"] > 0 or row["bruteforce_alerts"] > 0:
        return 1
    return 0

agg_features["high_risk_label"] = agg_features.apply(label_bucket, axis=1)

# Store final feature table
df_features = agg_features.copy()
df_features.head()


,ip,time_bucket_5min,total_events,login_failures,login_locked,distinct_users,suspicious_events,bruteforce_alerts,failure_ratio,suspicious_ratio,high_risk_label
0,10.0.0.10,2025-01-01 17:00:00,1,0,0,1,0,0.0,0.0,0.0,0
1,10.0.0.103,2025-01-01 11:30:00,1,0,0,1,0,0.0,0.0,0.0,0
2,10.0.0.103,2025-01-01 11:55:00,1,0,0,1,0,0.0,0.0,0.0,0
3,10.0.0.103,2025-01-01 12:25:00,1,0,0,1,0,0.0,0.0,0.0,0
4,10.0.0.105,2025-01-01 14:05:00,1,0,0,1,0,0.0,0.0,0.0,0


In [ ]:
# Train anomaly detection model with IsolationForest

# Import model from scikit-learn, widely used in security analytics and UEBA platforms
from sklearn.ensemble import IsolationForest

# Select feature columns for model training
feature_cols = [
    "total_events",
    "login_failures",
    "login_locked",
    "distinct_users",
    "failure_ratio",
    "suspicious_ratio",
]

# Build feature matrix
X = df_features[feature_cols].fillna(0)

# Initialize IsolationForest similar to anomaly modules in XDR tools
iso_model = IsolationForest(
    n_estimators=200,
    contamination=0.1,
    random_state=42,
)

# Fit model on aggregated security behavior
iso_model.fit(X)


IsolationForest(contamination=0.1, n_estimators=200, random_state=42)

In [ ]:
# Score security buckets and generate anomaly predictions

# Compute anomaly scores (lower scores mean more anomalous)
df_features["iso_score"] = iso_model.decision_function(X)

# Convert scores to binary anomaly label
df_features["iso_anomaly"] = iso_model.predict(X)

# Map IsolationForest output to human friendly label
df_features["iso_anomaly_label"] = df_features["iso_anomaly"].map(
    {1: "normal_behavior", -1: "anomalous_behavior"}
)

# Compare model perspective with high risk security label
df_features[["ip", "time_bucket_5min", "high_risk_label", "iso_score", "iso_anomaly_label"]].head(20)


,ip,time_bucket_5min,high_risk_label,iso_score,iso_anomaly_label
0,10.0.0.10,2025-01-01 17:00:00,0,0.0,normal_behavior
1,10.0.0.103,2025-01-01 11:30:00,0,0.0,normal_behavior
2,10.0.0.103,2025-01-01 11:55:00,0,0.0,normal_behavior
3,10.0.0.103,2025-01-01 12:25:00,0,0.0,normal_behavior
4,10.0.0.105,2025-01-01 14:05:00,0,0.0,normal_behavior
5,10.0.0.105,2025-01-01 14:30:00,0,0.0,normal_behavior
6,10.0.0.106,2025-01-01 10:35:00,0,0.0,normal_behavior
7,10.0.0.107,2025-01-01 12:20:00,0,0.0,normal_behavior
8,10.0.0.107,2025-01-01 13:45:00,0,0.0,normal_behavior
9,10.0.0.108,2025-01-01 13:55:00,0,0.0,normal_behavior


In [ ]:
# Build ranked anomaly dashboard for blue team

# Create ranking based on IsolationForest anomaly score
df_features["anomaly_rank"] = df_features["iso_score"].rank(ascending=True, method="first")

# Select most anomalous buckets for review, similar to XDR or UEBA console
TOP_N = 20
df_top_anomalies = df_features.sort_values("iso_score").head(TOP_N)

# Include key columns used by SOC analysts
cols_dashboard = [
    "ip",
    "time_bucket_5min",
    "total_events",
    "login_failures",
    "login_locked",
    "failure_ratio",
    "suspicious_ratio",
    "high_risk_label",
    "iso_score",
    "iso_anomaly_label",
    "anomaly_rank",
]
df_top_anomalies[cols_dashboard]


,ip,time_bucket_5min,total_events,login_failures,login_locked,failure_ratio,suspicious_ratio,high_risk_label,iso_score,iso_anomaly_label,anomaly_rank
996,203.0.113.10,2025-01-01 09:20:00,64,59,5,0.921875,1.0,1,-0.424610,anomalous_behavior,1.0
995,203.0.113.10,2025-01-01 09:15:00,53,47,6,0.886792,1.0,1,-0.423434,anomalous_behavior,2.0
994,203.0.113.10,2025-01-01 09:10:00,44,39,5,0.886364,1.0,1,-0.416922,anomalous_behavior,3.0
997,203.0.113.10,2025-01-01 09:25:00,39,37,2,0.948718,1.0,1,-0.413429,anomalous_behavior,4.0
509,10.0.1.220,2025-01-01 16:15:00,2,0,0,0.000000,0.0,0,-0.341354,anomalous_behavior,7.0
425,10.0.1.167,2025-01-01 15:50:00,2,0,0,0.000000,0.0,0,-0.341354,anomalous_behavior,6.0
150,10.0.0.207,2025-01-01 14:20:00,2,0,0,0.000000,0.0,0,-0.341354,anomalous_behavior,5.0
685,10.0.2.111,2025-01-01 15:05:00,2,0,0,0.000000,0.0,0,-0.341354,anomalous_behavior,8.0
196,10.0.0.24,2025-01-01 14:45:00,2,0,0,0.000000,0.0,0,-0.252363,anomalous_behavior,9.0
528,10.0.1.233,2025-01-01 12:25:00,2,0,0,0.000000,0.0,0,-0.252363,anomalous_behavior,10.0


In [ ]:
# Generate alert console view with context for zero trust and DevSecOps teams

# Merge top anomalies back to main events for detailed timeline
df_security_with_rank = df_security_events.merge(
    df_top_anomalies[["ip", "time_bucket_5min", "anomaly_rank"]],
    on=["ip", "time_bucket_5min"],
    how="left",
)

# Mark rows that belong to most suspicious buckets
df_security_with_rank["in_top_anomalies"] = df_security_with_rank["anomaly_rank"].notna()

# Apply zero trust idea and highlight events that require strong verification
df_security_with_rank["zero_trust_flag"] = np.where(
    df_security_with_rank["in_top_anomalies"] & (df_security_with_rank["role"] == "red_team"),
    "require_strong_auth_and_additional_checks",
    "standard_policy",
)

# Display condensed alert console similar to Splunk, Sentinel or Cortex XDR
alert_console_cols = [
    "timestamp",
    "ip",
    "user",
    "action",
    "label",
    "blue_team_action",
    "zero_trust_flag",
    "anomaly_rank",
]
df_security_with_rank[df_security_with_rank["in_top_anomalies"]][alert_console_cols].head(40)


,timestamp,ip,user,action,label,blue_team_action,zero_trust_flag,anomaly_rank
23,2025-01-01 09:10:01.023651,203.0.113.10,carol,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
25,2025-01-01 09:10:11.339360,203.0.113.10,carol,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
26,2025-01-01 09:10:19.553329,203.0.113.10,bob,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
28,2025-01-01 09:10:25.446098,203.0.113.10,bob,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
29,2025-01-01 09:10:26.882296,203.0.113.10,alice,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
30,2025-01-01 09:10:28.132362,203.0.113.10,bob,LOGIN_LOCKED,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
31,2025-01-01 09:10:42.549681,203.0.113.10,carol,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
32,2025-01-01 09:10:48.663339,203.0.113.10,carol,LOGIN_LOCKED,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
33,2025-01-01 09:10:49.659103,203.0.113.10,alice,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0
34,2025-01-01 09:11:02.013429,203.0.113.10,bob,LOGIN_FAILURE,suspicious,block_ip_and_force_password_reset,require_strong_auth_and_additional_checks,3.0


In [ ]:
# Define repository code snippets and metadata for security review

# Import regular expressions for pattern search
import re

# Create sample code base representing microservices and infrastructure scripts
code_repo = [
    {
        "file": "services/auth_service/auth.py",
        "service": "auth_service",
        "language": "python",
        "code": """
API_KEY = "AKIA1234567890000000"
DB_PASSWORD = "SuperSecret123"
def login(user, password):
    # TODO: validate input
    print("User logged:", user)
""",
    },
    {
        "file": "services/payment_service/payment.js",
        "service": "payment_service",
        "language": "javascript",
        "code": """
const crypto = require('crypto');
const secret = "plain-text-secret";
function hashCard(card) {
    return crypto.createHash('md5').update(card).digest('hex');
}
""",
    },
    {
        "file": "infra/provisioning/terraform.tf",
        "service": "infra",
        "language": "hcl",
        "code": """
resource "aws_security_group" "web_sg" {
  name        = "web_sg_open"
  description = "Open security group for web"
  ingress {
    from_port   = 0
    to_port     = 65535
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
}
""",
    },
]


In [ ]:
# Scan code base for insecure patterns with SAST-style rules

# Define rules similar to SonarQube, Checkmarx, Bandit or Snyk checks
sast_rules = [
    {"id": "HARDCODED_PASSWORD", "pattern": r"password\s*=", "severity": "high"},
    {"id": "HARDCODED_SECRET", "pattern": r"secret\s*=", "severity": "high"},
    {"id": "PLAINTEXT_API_KEY", "pattern": r"AKIA[0-9A-Z]{16}", "severity": "high"},
    {"id": "WEAK_HASH_MD5", "pattern": r"md5", "severity": "medium"},
    {"id": "OPEN_SG_0_0_0_0", "pattern": r"0\.0\.0\.0/0", "severity": "high"},
]

# Create list to store findings per file
sast_findings = []

# Iterate over repository files and apply static analysis style rules
for item in code_repo:
    for rule in sast_rules:
        matches = re.findall(rule["pattern"], item["code"], flags=re.IGNORECASE)
        if matches:
            sast_findings.append(
                {
                    "file": item["file"],
                    "service": item["service"],
                    "language": item["language"],
                    "rule_id": rule["id"],
                    "severity": rule["severity"],
                    "match_count": len(matches),
                }
            )

# Convert findings to DataFrame for DevSecOps dashboards
df_code_findings = pd.DataFrame(sast_findings)
df_code_findings


,file,service,language,rule_id,severity,match_count
0,services/auth_service/auth.py,auth_service,python,HARDCODED_PASSWORD,high,1
1,services/auth_service/auth.py,auth_service,python,PLAINTEXT_API_KEY,high,1
2,services/payment_service/payment.js,payment_service,javascript,HARDCODED_SECRET,high,1
3,services/payment_service/payment.js,payment_service,javascript,WEAK_HASH_MD5,medium,1
4,infra/provisioning/terraform.tf,infra,hcl,OPEN_SG_0_0_0_0,high,1


In [ ]:
# Aggregate SAST findings per service and severity

# Group findings similar to SonarQube or GitHub Advanced Security dashboards
svc_agg = (
    df_code_findings.groupby(["service", "severity"])
    .agg(total_issues=("rule_id", "count"))
    .reset_index()
)

# Pivot table to get columns per severity
svc_pivot = svc_agg.pivot_table(
    index="service",
    columns="severity",
    values="total_issues",
    fill_value=0,
)

# Ensure consistent columns for gate logic
for sev in ["high", "medium", "low"]:
    if sev not in svc_pivot.columns:
        svc_pivot[sev] = 0

svc_pivot = svc_pivot.reset_index()
svc_pivot


severity,service,high,medium,low
0,auth_service,2.0,0.0,0
1,infra,1.0,0.0,0
2,payment_service,1.0,1.0,0


In [ ]:
# Apply DevSecOps-style security gate with shift-left mindset

# Define thresholds similar to CI security jobs in GitLab CI or GitHub Actions
MAX_HIGH_ALLOWED = 0
MAX_MEDIUM_ALLOWED = 2

# Create gate status per service
def compute_gate_status(row):
    # Block release when high severity issues exist
    if row["high"] > MAX_HIGH_ALLOWED:
        return "fail_gate_block_release"
    # Warn but allow when only few medium issues exist
    if row["medium"] > MAX_MEDIUM_ALLOWED:
        return "warn_and_require_fix_soon"
    return "pass_gate"

svc_pivot["security_gate_status"] = svc_pivot.apply(compute_gate_status, axis=1)

# Add field to highlight shift-left action required
svc_pivot["shift_left_action"] = svc_pivot["security_gate_status"].map(
    {
        "fail_gate_block_release": "stop_pipeline_and_fix_critical_issues",
        "warn_and_require_fix_soon": "schedule_fix_and_add_unit_tests",
        "pass_gate": "allow_deploy_and_continue_monitoring",
    }
)

svc_pivot


severity,service,high,medium,low,security_gate_status,shift_left_action
0,auth_service,2.0,0.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues
1,infra,1.0,0.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues
2,payment_service,1.0,1.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues


In [ ]:
# Map runtime events to services for unified risk view

# Define routing logic to associate ip ranges with services
def map_ip_to_service(ip):
    if ip.startswith("10.0.0."):
        return "auth_service"
    if ip.startswith("10.0.1."):
        return "payment_service"
    if ip.startswith("10.0.2."):
        return "infra"
    if ip == "203.0.113.10":
        return "auth_service"
    return "unknown"

# Add service column to runtime security events
df_security_with_rank["service"] = df_security_with_rank["ip"].apply(map_ip_to_service)

# Aggregate runtime risk per service similar to SOC dashboards
runtime_risk = (
    df_security_with_rank.groupby("service")
    .agg(
        total_events=("action", "count"),
        suspicious_events=("label", lambda x: (x == "suspicious").sum()),
        bruteforce_alerts=("alert_bruteforce", "sum"),
        top_anomaly_buckets=("in_top_anomalies", lambda x: x.sum()),
    )
    .reset_index()
)

runtime_risk


,service,total_events,suspicious_events,bruteforce_alerts,top_anomaly_buckets
0,auth_service,526,200,200.0,204
1,infra,334,0,0.0,12
2,payment_service,340,0,0.0,6


In [ ]:
# Build unified DevSecOps risk dashboard per service

# Merge code-level gate status with runtime security metrics
service_risk = runtime_risk.merge(
    svc_pivot[["service", "high", "medium", "low", "security_gate_status", "shift_left_action"]],
    on="service",
    how="left",
)

# Compute combined risk score joining code and runtime factors
service_risk["combined_risk_score"] = (
    service_risk["suspicious_events"]
    + service_risk["bruteforce_alerts"] * 2
    + service_risk["high"] * 3
    + service_risk["medium"] * 2
)

# Add recommendation for DevSecOps and SOC teams
def recommend_next_action(row):
    # Prioritize services with high combined risk and failed gate
    if row["security_gate_status"] == "fail_gate_block_release":
        return "prioritize_hotfix_and_block_new_deployments"
    if row["combined_risk_score"] > 5:
        return "schedule_security_sprint_and_improve_monitoring"
    return "keep_current_controls_and_review_regularly"

service_risk["devsecops_recommendation"] = service_risk.apply(recommend_next_action, axis=1)

service_risk.sort_values("combined_risk_score", ascending=False)


,service,total_events,suspicious_events,bruteforce_alerts,top_anomaly_buckets,high,medium,low,security_gate_status,shift_left_action,combined_risk_score,devsecops_recommendation
0,auth_service,526,200,200.0,204,2.0,0.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues,606.0,prioritize_hotfix_and_block_new_deployments
2,payment_service,340,0,0.0,6,1.0,1.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues,5.0,prioritize_hotfix_and_block_new_deployments
1,infra,334,0,0.0,12,1.0,0.0,0,fail_gate_block_release,stop_pipeline_and_fix_critical_issues,3.0,prioritize_hotfix_and_block_new_deployments
